In [24]:
from string import punctuation
with open("sherlock.txt", "r") as f:
    text = f.read()


In [25]:
# replaces new lines with full stops
text = text.replace('\n', '.').lower()

In [28]:
# splits it up into sentences
sentences = text.split(".")


['i', ' a scandal in bohemia', '', '', 'i', '', '', 'to sherlock holmes she is always _the_ woman', ' i have seldom heard him', 'mention her under any other name']


In [36]:
# gets rid of empty stirngs and get rid of the "i" for chapter numbers
sentences = list(filter(None, sentences))
for counter, sent in enumerate(sentences):
    if sent == "i":
        del sentences[counter]


In [37]:
print(sentences[:10])

[' a scandal in bohemia', 'to sherlock holmes she is always _the_ woman', ' i have seldom heard him', 'mention her under any other name', ' in his eyes she eclipses and', 'predominates the whole of her sex', ' it was not that he felt any emotion', 'akin to love for irene adler', ' all emotions, and that one particularly,', 'were abhorrent to his cold, precise but admirably balanced mind']


So I want the table to look like:
text | length | how many letters are used (uniqueness) | maybe the chi squared score ? (normalised distribution not english) | frequency distribution | what it is 

I want it to include these things:
base64
sha1
md5
sha256
caeser cipher
plaintext